## Set-up

Install

In [22]:
# Update to requirements.txt
# !pip install 
# !pip install kaggle
# !kaggle datasets download -d chitwanmanchanda/fraudulent-transactions-data -p data --unzip

100%|███████████████████████████████████████▊| 177M/178M [00:13<00:00, 14.5MB/s]
100%|████████████████████████████████████████| 178M/178M [00:13<00:00, 14.2MB/s]


Methods

In [48]:
import pandas as pd
import networkx as nx
import graphistry
import cudf
import cugraph

# Custom
# from scripts.config import DATA_DIR

In [ ]:
# Add cuda check

### Data Wrangling and Featurization

Find a (raw) rich dataset in fraud/cybersecurity/recommendation/supply chain with some notion of events & entities (logs, orders / bill of materials, ..), pull it in, clean and load it. 

In [45]:
fraud.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [41]:
%%time
M = nx.MultiDiGraph()

CPU times: user 2.28 s, sys: 440 ms, total: 2.72 s
Wall time: 2.71 s


In [42]:
%%time
print(f"Nodes: {M.number_of_nodes():,}")
print(f"Edges: {M.number_of_edges():,}")

Nodes: 0
Edges: 0
CPU times: user 104 µs, sys: 8 µs, total: 112 µs
Wall time: 85.8 µs


What are good columns for features? Why?

Perform feature engineering. Be ready to discuss data quality, insights, & challenges.

What are possible insights?
Is the dataset balanced?
What distributions likely model the different columns?
Is there a target(s)? Why are they important?
Descriptive statistics?

## GNN

Turn the dataset into a GNN model - Popular ones include GCN, GAT, RGCN

Depending on the dataset, fit one (or more) of the following models:
- Link Prediction
- Node Prediction
- Graph Classification
- Motif Mining

How does the model perform? How would you improve it with more time?

#### GNN Visualization $\rightarrow$ see `viz.ipynb` notebook

## Extra credit

Can you wrap the above in basic MLOps? 

How about model explainability? 